# Exploring Rules Extraction for Sport Image Classification

**Introduction:**

Welcome to HES-Xplain, our interactive platform designed to facilitate explainable artificial intelligence (XAI) techniques. In this `demo`, we dive into sport image classification and showcase the power of Rules Extraction as an interpretability tool.

Rules Extraction allow us to reduce the complexity of the CNN (fully-connected layers) to a small set of relevant rules, without a great loss in accuracy. Furthermore, these rules can be interpreted by looking at how they split the input data. Such a combined analysis helps us to better understand the global behavior of the network.

The library implementation and this demo mostly follow the paper [Improving neural network interpretability via rule extraction](https://arodes.hes-so.ch/record/4647?ln=fr). And aim at replacing the fully-connected layers of a Convolutional Neural Network (CNN) with a small set of rules, allowing for better interpretation of its decisions while preserving accuracy. It create Ensemble Rule based binary classifier that is able to discriminate between a target class versus others. 

With our [rules-extraction library](https://github.com/HES-XPLAIN/rules-extraction), you can effortlessly extract rules from a CNN model and seamlessly utilize this ensemble of rules in a manner compatible with the scikit-learn API. You can also create plot that illustrate the data and rules. 



---
## Google Colab Setup

This section prepares the notebook for use with Google Colaboratory. If applicable, change the following variable to True:



In [ ]:
# Colab compatibility
use_colab = False

In [ ]:
if use_colab:
    # ensure the directory is empty
    !rm -rf * .config

    !# install codebase from GitHub
    !git clone --no-checkout https://github.com/HES-XPLAIN/notebooks.git --depth=1 .
    !git config core.sparseCheckout true
    !git sparse-checkout set --cone
    !git sparse-checkout add use_case_sport_classification
    !git sparse-checkout reapply
    !git checkout main

    # adjust folder structure
    !mv use_case_sport_classification/* .
    !rm -rf use_case_sport_classification/

In [ ]:
if use_colab:
    # Install dependencies
    !pip install rules_extraction

When asked to, restart the session by clicking on the "Restart Session" button, then continue and execute the next cells.

## Workspace Setup

This section download the required code and models from our GitHub and huggingface.co repositories.

Download the dataset and models.

In [ ]:
# download and extract dataset
import zipfile
from huggingface_hub import hf_hub_download

REPO_ID = "HES-XPLAIN/SportsImageClassification"
FILENAME = "SportsImageClassification.zip"
dataset_file_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
extract_path = 'data'

# Unzip the file
with zipfile.ZipFile(dataset_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset successfully extracted to: {extract_path}")

In [ ]:
!mkdir -p models_weight
!curl -OL https://huggingface.co/HES-XPLAIN/sport_classification/resolve/main/VGGSportsImageClassification.pth -o VGGSportsImageClassification.pth
!mv *.pth models_weight/

## Dataset and Problem Statement

The sport image classification dataset used in this use case consists of approximately 15,000 images, covering 100 different sports categories. This dataset, available on [Kaggle](https://www.kaggle.com/datasets/gpiosenka/sports-classification), contains a total of 13,493 train images, 500 test images, and 500 validation images. The images are in JPG format and have dimensions of 224x224 pixels with 3 color channels.

**Download dataset**: To make a propoer use of this demo, the dataset will be automatically downloaded and extracted in the data folder. You can download it using the next cell or from our [Hugging Face repository](https://huggingface.co/datasets/HES-XPLAIN/SportsImageClassification).

**Problem Statement**: Our objective is to build a robust image classifier capable of accurately classifying sport images among the 100 predefined sports categories. The nature of the dataset, comprising images of various sports, makes it particularly suitable for introducing and visualizing XAI techniques.

In [ ]:
# global imports 
import sys
import os

from rules_extraction.rules import Rule, EnsembleRule, RuleRanker
from rules_extraction.plot import plot_accuracy, plot_frontier
from rules_extraction.utils import *

from scripts.models import FineTunedVGG
from scripts.custom_dataset import CustomDataset
from scripts.helpers import *

import torch
import json
import torch.nn as nn
import pandas as pd

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, Subset

## Loading a CNN model

**Model Weights**: We provide weights for a fine-tuned VGG on the SportsImageDataset. You can download it using the next cell or from our [Hugging Face repository](https://huggingface.co/HES-XPLAIN). While using Github Codespace you can have direct access to the weight in the folder `./models_weight`.

**Loading Weights** : By utilizing the functions and classes defined in our helpers file, you can initialize a VGG model and load the weights.

---
Utilize GPU and CUDA for faster processing speed and load the weight

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (GPU)")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) not available. Using CPU.")

# load trained model VGG
model = FineTunedVGG()
model = model.to(device)
model.eval()

# Specify the map_location argument when loading the model
load_path = "models_weight/VGGSportsImageClassification.pth"
checkpoint = torch.load(load_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])

# get loaders  using function define in scripts for this notebook
train_loader, valid_loader, test_loader = get_dataloaders()

# get a dictionary that links class names to integers, making it easier for us as we proceed in the notebook
with open('./data/idx_to_names.json', 'r') as file:
    idx_to_names = json.load(file)
names_to_idx = {v: k for k, v in idx_to_names.items()}

## Rules extraction

From this point, we'll utilize class and functions from the rules-extraction python library, accessible on our [HES-XPLAIN platform](https://github.com/HES-XPLAIN) and the [pypi index](https://pypi.org/project/rules-extraction).

**Filter dataset**: Let's create a filtered version of the data. To do this we will initialize a dataloader using only the subset of data that are correctly classified by the CNN model. This is not mandatory but it allow us to establish the CNN's performane as the ground truth. This step enable us to assess the extent of deviation between our simplification and the standard model. We can use the `rules_extraction.utils.filter_dataset` function to get the list of well classified data index. From this we can initialize a new loader.

**Compute image representation**: The rules are derived from an image representation generated by the model, specifically the final feature map of the CNN. This feature map is averaged to yield a vector, providing a condensed representation of the image. The functions `rules_extraction.utils.compute_avg_features` implement this logic and returns a dataframe.

**Create a target class dataset**: The Ensemble Rules model will be able to perform binary classification, thus we need to parse the dataset to create a binary dataset containing N datapoints of target class and N of random other class. This allow to have a balance dataset.

**Extract rules**: Once we got the image representation we can extract rules from a RandomForestClassifier model using the `rules_extraction.utils.extract_all_rules` function. You have the flexibility to pass parameters to customize your Random Forest model as needed. We recommend using `max_depth=2` to ensure the rules remain understandable.

**Rank Rules**: To reduce complexity we can rank rules and we will later use only some of the top ranked rules to. This method use a simple percpetron and use weight to classify rules importance. The logic is encapsulated in the `rules_extraction.rules.RuleRanker.rank_rules` method.l.

In [ ]:
# create filtered loaders

model = model
loader = test_loader
device = device 

# test set
correct_test_idx = filter_dataset(model=model, loader=loader, device=device)
test_filtered_dataset = Subset(test_loader.dataset, correct_test_idx)
test_filtered_dataloader = DataLoader(dataset=test_filtered_dataset, batch_size=test_loader.batch_size, shuffle=False)

loader = train_loader
#train set
correct_train_idx = filter_dataset(model=model, loader=loader, device=device)
train_filtered_dataset = Subset(train_loader.dataset, correct_train_idx)
train_filtered_dataloader = DataLoader(dataset=train_filtered_dataset, batch_size=train_loader.batch_size, shuffle=False)

In [ ]:
# compute avg features map 

test_features = compute_avg_features(model, test_filtered_dataloader, idx_to_names, device)
train_features = compute_avg_features(model, train_filtered_dataloader, idx_to_names, device)

test_features

As you can see, our dataframe now represents each image with a vector of 512 floats. These vectors are obtained by averaging the last feature map of VGG.

---
As mentioned earlier, our current task involves binary classification, specifically targeting the classification of one specific category against the rest. To achieve this, we must dynamically parse our dataframe and generate a binary dataset on the fly. 

For this, we select a target class of interest and employ the function rules_extraction.utils.make_target_df. This function returns a dataframe containing all the data for the target class, while also sampling from other classes to create a balanced dataset.

Let's investigate the 'air hockey' class. 

In [ ]:
df_test = make_target_df(test_features, 'air hockey')
df_train = make_target_df(train_features, 'air hockey')
X_train, y_train = df_train.iloc[:, :-3], df_train.iloc[:, -1]
X_test, y_test = df_test.iloc[:, :-3], df_test.iloc[:, -1]

In [ ]:
# extract all rules from a RandomForestClassifier
all_rules = extract_all_rules(X_train, y_train, n_estimators=200, max_depth=2, random_state=1)

In [ ]:
# rank rules extract only the top 30 
top_30_rules = RuleRanker(all_rules, X_train, y_train).rank_rules(N=30)

## Create rule based model 

**Ensemble Rule Modeling**: 

Once the rules are ranked, we can make use of our rule classifier API. To begin, we initialize individual rules using our simple rule class and then select the top N rules to initialize the EnsembleRule classifier.

Both the Rule and EnsembleRule are designed to be compatible with the scikit-learn API. They implement the .fit, .predict, and .score methods.

In [ ]:
# Extract conditions and labels from the list and initialize a list of Rule
conditions_list, labels_list = zip(*top_30_rules)
rules = [Rule(conditions, label) for conditions, label in zip(conditions_list, labels_list)]

# Initialize ensemble rule
ensemble_rule = EnsembleRule(rules)

# Predict with ensemble rule
data_point = df_test.iloc[0]
ensemble_prediction = ensemble_rule.predict(data_point)
print(f"Ensemble Rule Prediction: {ensemble_prediction[0]} the correct prediction is {data_point['binary_label']}")

## Rules Extraction results and plots.

**Prediction and score**: We can use ensemble.predict or rule.predict to make prediction on one or multiple data points and use the ensemble.score or rule.score method to compute the accuracy. 

**Plots**: The `rules_extraction.plot` API enable two plots. 

1. Plot accuracy: where you can compute and plot the accuracy on a dataset versus the number of rules used. This show how adding rules improve the model. Usually we can see that from two to three rules our EnsembleRule classifier is able to achieve 80-90% accuracy.

2. Plot frontier: this plot show a rule for one class with example images and plot the rule's frontier. Images are embedded according to their average filter activation. The most characteristic images of the class are grouped together as they have similar filter activations. We can also get the intuition for visual patterns that are present in one class but absent in all other classes.

In [ ]:
ensemble_predictions = ensemble_rule.predict(X_test)
print(f"Ensemble Rule predictions on test set are: {ensemble_predictions}. And the correct predictions are {y_test.tolist()}")
print(f"Accuracy on test set: {ensemble_rule.score(X_test, y_test):.3f}")

---
Let's do some plots now: the `rules_extraction.plot` API enable two plots. 

1. Plot accuracy: where you can compute and plot the accuracy on a dataset versus the number of rules used. This show how adding rules improve the model. Usually we can see that from two to three rules our EnsembleRule classifier is able to achieve 80-90% accuracy.

2. Plot frontier: this plot show a rule for one class with example images and plot the rule's frontier. Images are embedded 
according to their average filter activation The most characteristic images of the class are grouped together as they have 
similar filter activations. We can also get th  intuition for visual patterns tha 
are present in one class but absent in all other classes..

In [ ]:
plot_accuracy(rules, df_train, class_name=None, n=9, save_path=None)
plot_accuracy(rules, df_test, class_name=None, n=9, save_path=None)


In [ ]:
plot_frontier(train_features, rule=rules[1], target_class='air hockey', model=model, alpha=0.55, device=device)

## Conclusion:

Throughout this notebook, we demonstrate the application of our rules-extraction library for extracting rules from a convolutional model. Additionally, we illustrate how to use these rules to make predictions employing a simpler and more interpretable model.

Utilizing an ensemble of rule-based models empowers users to have control over their predictions and offers potential explanations. Although average features may not be directly understandable by humans, they contribute to reducing the complexity of a CNN. Further exploration can be undertaken to delve into the significant features, including those used by the top rule.

Furthermore, by employing the frontier plot, we aim to gain intuition into visual patterns that are present in one class but absent in all other classes. This approach contributes to enhancing the explainability of the CNN model. 

## References

1. [Rule Extraction paper](https://arodes.hes-so.ch/record/4647?ln=fr)
2. [Scikit-learn](https://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html)
